# 데이터 크롤링

In [1]:
import urllib.request as req
import gzip, os, os.path
savepath = "./mnist"
baseurl = "http://yann.lecun.com/exdb/mnist"
files = [
    "train-images-idx3-ubyte.gz",
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz",
    "t10k-labels-idx1-ubyte.gz"]
# 다운로드
if not os.path.exists(savepath): os.mkdir(savepath)
for f in files:
    url = baseurl + "/" + f
    loc = savepath + "/" + f
    print("download:", url)
    if not os.path.exists(loc):
        req.urlretrieve(url, loc)
# GZip 압축 해제
for f in files:
    gz_file = savepath + "/" + f
    raw_file = savepath + "/" + f.replace(".gz", "")
    print("gzip:", f)
    with gzip.open(gz_file, "rb") as fp:
        body = fp.read()
        with open(raw_file, "wb") as w:
            w.write(body)
print("ok")

download: http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
gzip: train-images-idx3-ubyte.gz
gzip: train-labels-idx1-ubyte.gz
gzip: t10k-images-idx3-ubyte.gz
gzip: t10k-labels-idx1-ubyte.gz
ok


In [2]:
import struct  # byte 단위로 끊어서 이동해주는 module
def to_csv(name, maxdata):
    # 레이블 파일과 이미지 파일 열기
    lbl_f = open("./mnist/"+name+"-labels-idx1-ubyte", "rb")
    img_f = open("./mnist/"+name+"-images-idx3-ubyte", "rb")
    csv_f = open("./mnist/"+name+".csv", "w", encoding="utf-8")

    # 헤더 정보 읽기 --- (※1)      
    #'>||'' : 이동 , f.read(8) : 8byte 읽기 , ( mag, lbl_count) : 4byte 씩 끊어서 할당

    mag, lbl_count = struct.unpack(">II", lbl_f.read(8))
    mag, img_count = struct.unpack(">II", img_f.read(8))
    rows, cols = struct.unpack(">II", img_f.read(8))
    pixels = rows * cols
    # 이미지 데이터를 읽고 CSV로 저장하기 --- (※2)
    res = []
    for idx in range(lbl_count):
        if idx > maxdata: break
        label = struct.unpack("B", lbl_f.read(1))[0]
        bdata = img_f.read(pixels)
        sdata = list(map(lambda n: str(n), bdata))
        csv_f.write(str(label)+",")
        csv_f.write(",".join(sdata)+"\r\n")
        # 잘 저장됐는지 이미지 파일로 저장해서 테스트하기 -- (※3)
        if idx < 10:
            s = "P2 28 28 255\n"
            s += " ".join(sdata)
            iname = "./mnist/{0}-{1}-{2}.pgm".format(name,idx,label)
            with open(iname, "w", encoding="utf-8") as f:
                f.write(s)
    csv_f.close()
    lbl_f.close()
    img_f.close()
# 결과를 파일로 출력하기 --- (※4)
to_csv("train", 1000)
to_csv("t10k", 500)

# mnist to_ csv

In [3]:
from sklearn import model_selection, svm, metrics
# 분류모형 평가지표 metrics

In [12]:
def load_csv(fname):
    
    labels =[ ] # label
    images = [ ] # features
    
    with open(fname, 'r') as f:
        
        for line in f:
            cols = line.split(',')
            if len(cols) < 2: continue
            
#             파일을 읽을 때 무조건 문자열로 읽음. 따라서 int 변환 필수
            labels.append(int(cols.pop(0)))
#             SVM은 feature scaling(정규화 작업)된 data를 학습할 때 더 정교하다는 이론이 있음
            vals = list( map( lambda n: int(n) / 256, cols ) )
            images.append(vals)
    return {'labels':labels,'images':images}
                



train = load_csv('./mnist/train.csv')
test = load_csv('./mnist/t10k.csv')

##### 데이터 json 형태 저장

# 모델 훈련

In [15]:
# SVR // SVC(분류형)

clf = svm.SVC()
clf.fit(train['images'], train['labels'])

C:\Users\student\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [17]:
predict = clf.predict(test['images'])

ac_score = metrics.accuracy_score(test['labels'],predict)
print('정답률 =', ac_score)

cl_report = metrics.classification_report(test["labels"], predict)
print('리포트 =')
print(cl_report)

정답률 = 0.7884231536926147
리포트 =
              precision    recall  f1-score   support

           0       0.87      0.93      0.90        42
           1       0.81      1.00      0.89        67
           2       0.84      0.69      0.76        55
           3       0.87      0.57      0.68        46
           4       0.76      0.75      0.75        55
           5       0.63      0.80      0.71        50
           6       0.97      0.67      0.79        43
           7       0.74      0.86      0.79        49
           8       0.91      0.72      0.81        40
           9       0.71      0.81      0.76        54

    accuracy                           0.79       501
   macro avg       0.81      0.78      0.78       501
weighted avg       0.80      0.79      0.79       501



### 1. 데이터 수집<br> 2. 전처리(무조건 벡터화) --> 2차원( matrix )<br> 3. 학습모형<br><br>(1)지도학습<br><br>- 분류 ==> 평가<br>- 수치예측<br><br>(2)비지도학습<br><br>4. 모델 평가